# LAB 7: Error analysis

Objectives
* Construct a  linear text classifier using SGDClassifier
* Evaluate its performance and categorize the errors that it makes
* Eaxmine model's coefficients and decision function values
* Interpret model results using LIME

In [ ]:
import numpy as np
import pandas as pd
from cytoolz import *
from tqdm.auto import tqdm

tqdm.pandas()

---

## Load data

In [ ]:
train = pd.read_parquet("s3://ling583/lab7-train.parquet", storage_options={"anon":True})
test = pd.read_parquet("s3://ling583/lab7-test.parquet", storage_options={"anon":True})

In [ ]:
import cloudpickle
from sklearn.metrics import classification_report, f1_score, plot_confusion_matrix

In [ ]:
sgd = cloudpickle.load(open("sgd.model", "rb"))

In [ ]:
predicted = sgd.predict(test["text"])
print(classification_report(test["topics"], predicted))

---

## Decision function

In [ ]:
labels = sgd.named_steps['sgdclassifier'].classes_
scores = sgd.decision_function(test["text"])

In [ ]:
labels

In [ ]:
scores[0,:]

In [ ]:
test['topics'].iloc[0]

In [ ]:
test['text'].iloc[0]

In [ ]:
highest = scores.max(axis=1)

In [ ]:
highest[0]

In [ ]:
highest.argmax()

In [ ]:
scores[?]

In [ ]:
test['topics'].iloc[?]

In [ ]:
test['text'].iloc[?]

In [ ]:
highest.argmin()

In [ ]:
scores[?]

In [ ]:
test['topics'].iloc[?]

In [ ]:
test['text'].iloc[?]

In [ ]:
scores[0:5,:]

In [ ]:
scores.sort(axis=1)
scores[0:5,:]

In [ ]:
margin = scores[:,3]-scores[:,2]

In [ ]:
margin.max(), margin.min()

In [ ]:
print(classification_report(test["topics"][margin > 5], predicted[margin > 5]))

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
thresh = np.linspace(-2, 3, 50)
x = [100*(1-sum(margin > t)/len(margin)) for t in thresh]
y = [f1_score(test["topics"][margin > t], predicted[margin > t], average="macro") for t in thresh]
plt.plot(x, y)
plt.xlabel('% Unknowns')
plt.ylabel('F1')

In [ ]:
print(classification_report(test["topics"][margin > 1.75], predicted[margin > 1.75]))

**TO DO:** Summarize your results for this section. What could we do if we wanted to make label as many examples as possible while still keeping F1 above 0.99?

---

## Model parameters

In [ ]:
coef = sgd.named_steps['sgdclassifier'].coef_
labels, coef

In [ ]:
coef.shape

In [ ]:
vocab = sgd.named_steps['countvectorizer'].get_feature_names()

In [ ]:
coef[0,:].argmax()

In [ ]:
coef[0,13100]

In [ ]:
vocab[13100]

In [ ]:
ranked = np.argsort(coef, axis=1)
for i, label in enumerate(labels):
    print(label)
    for j in concat([range(-1, -11, -1), range(10, 0, -1)]):
        print(f'  {vocab[ranked[i,j]]:15s} {coef[i, ranked[i,j]]:6.3f}')
    print()

In [ ]:
from kwic import kwic

In [ ]:
kwic('newsroom', train['text'])

In [ ]:
kwic('zaire', train['text'])

**TO DO:** What can you conclude about the model from looking at the coefficients? Is there evidence of overfitting? How could we improve the results?